In [47]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import re
import sqlite3
import os

In [48]:
# headers = ({'User-Agent':
#             'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
#             'Accept-Language': 'en-US, en;q=0.5'})
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
           "Accept-Encoding":"gzip, deflate", 
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
           "DNT":"1","Connection":"close", 
           "Upgrade-Insecure-Requests":"1"}
main_page_urls = ["https://www.amazon.com/Beyond-Meat-Beef-16-Ounce/dp/B07R6TRHQ2/"]

In [49]:
def scrape_reviews(main_page_url, headers):
    main_page = requests.get(main_page_url, headers=headers)
    main_page_soup = BeautifulSoup(main_page.content, 'html.parser')
    time.sleep(2)

    data = []
    
    try:
        product_name = main_page_soup.find('span', {'id': 'productTitle'}).get_text().strip()
    except Exception as e:
        product_name = ""
        print(e)
        
    try:
        brand_name = main_page_soup.find('a', {'id': 'bylineInfo'}).get_text().strip()[7:]
    except Exception as e:
        brand_name = ""
        print(e)
    
    try:
        price = main_page_soup.find('span', {'class': 'priceBlockBuyingPriceString'}).get_text().strip()[1:]
    except Exception as e:
        price = ""
        print(e)
        
    try:
        asin = main_page_soup.find('ul', {'class': 'detail-bullet-list'}).findAll('li')[4].findAll('span')[2].get_text()
    except Exception as e:
        asin = ""
        print(e)
    
    try:
        overall_rating = main_page_soup.find('span', {'data-hook': 'rating-out-of-text'}).get_text()[:3]
    except Exception as e:
        overall_rating = ""
        print(e)
    
    review_url_base = main_page_url[:-1].replace("dp", "product-reviews") + "?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber="
    review_urls = [review_url_base + str(i) for i in range(1, 10)]
    
    for review_url in review_urls:
        review_page = requests.get(review_url, headers=headers)
        review_page_soup = BeautifulSoup(review_page.content, 'html.parser')
        time.sleep(2)
        username = review_page_soup.findAll('span', {'class': 'a-profile-name'})
        date = review_page_soup.findAll('span', {'data-hook': 'review-date'})
        rating = review_page_soup.find_all('i', {'data-hook': 'review-star-rating'})
        review = review_page_soup.find_all('span', {'data-hook': 'review-body'})
        
        for i in range(2, 8):
            try:
                reviewer_username = username[i].get_text()
                review_date = date[i].get_text()[33:]
                review_rating = rating[i].span.get_text()[:3]
                review_text = review[i].get_text()[:-9].strip()

                data.append([brand_name, product_name, asin, price, overall_rating, reviewer_username, review_date, review_rating, review_text])
                
            except Exception as e:
                print(e)
                continue
    return data

In [50]:
def write_to_db(data):
    for entry in data:

        conn = sqlite3.connect("reviewdb.db")
        c = conn.cursor()
        c.execute("""CREATE TABLE IF NOT EXISTS brand_table('index_brand' INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
                                                            'name_brand' VARCHAR(100));""")
        c.execute("""CREATE TABLE IF NOT EXISTS product_table('index_product' INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                                                            'index_brand_product' INTEGER,
                                                            'asin_product' VARCHAR(30),
                                                            'name_product' VARCHAR(1000), 
                                                            'price_product' FLOAT, 
                                                            'overall_rating_product' FLOAT,
                                                            FOREIGN KEY (index_brand_product) REFERENCES brand_table (index_brand));""")
        c.execute("""CREATE TABLE IF NOT EXISTS review_table('index_review' INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                                                        'index_product_review' INTEGER, 
                                                        'rating_review' INTEGER,
                                                        'reviewer_id_review' VARCHAR(100), 
                                                        'date_review' VARCHAR(100), 
                                                        'text_review' VARCHAR(2000), 
                                                        'length_review' FLOAT,
                                                        FOREIGN KEY (index_product_review) REFERENCES product_table (index_product));""")

        c.execute("SELECT index_brand FROM brand_table WHERE name_brand == ? ;", (entry[0], ))
        index_brand=c.fetchall()

        if index_brand == []:
            c.execute("INSERT INTO brand_table VALUES ( ?, ? );", (None, entry[0]))
            c.execute("SELECT index_brand FROM brand_table WHERE name_brand ==  ? ;", (entry[0],))
            index_brand=c.fetchall()



        c.execute("SELECT index_product FROM product_table WHERE asin_product == ? ;", (entry[2],))
        index_product=c.fetchall()
        if index_product == []:
            c.execute("""INSERT INTO product_table ('index_brand_product',
                                                        'asin_product',
                                                        'name_product', 
                                                        'price_product', 
                                                        'overall_rating_product') VALUES (?, ?, ?, ?, ?);""",
            (index_brand[0][0], entry[2], entry[1], float(entry[3]), float(entry[4])))
            c.execute("SELECT index_product FROM product_table WHERE asin_product == ? ;", (entry[2],))
            index_product=c.fetchall()

        c.execute("SELECT text_review FROM review_table WHERE text_review == ? ;", (entry[8],))
        text_review=c.fetchall()
        if text_review == []:
            c.execute("""INSERT INTO review_table ('index_product_review', 
                                                            'rating_review',
                                                            'reviewer_id_review', 
                                                            'date_review', 
                                                            'text_review', 
                                                            'length_review') VALUES (?, ?, ?, ?, ?, ?);""", 
            (index_product[0][0], int(entry[7][0]), entry[5], entry[6], entry[8], len(entry[8].split(" "))))

        conn.commit()
        conn.close()

In [51]:
for main_page_url in main_page_urls:
    data = scrape_reviews(main_page_url, headers)
    write_to_db(data)

In [52]:
def select_all():
    conn = sqlite3.connect("reviewdb.db")
    c = conn.cursor()
    c.execute("SELECT * FROM brand_table")
    output1 = c.fetchall()
    c.execute("SELECT * FROM product_table")
    output2 = c.fetchall()
    c.execute("SELECT * FROM review_table")
    output3 = c.fetchall()
    conn.close()
    print(output1)
    print(output2)
    print(output3)
select_all()

[(1, 'Beyond Meat')]
[(1, 1, '', 'Beyond Meat from PlantBased Frozen oz lb. Package, Ground Beef Substitute, 16 Ounce', 10.49, 4.6), (2, 1, 'USA', 'Beyond Meat from PlantBased Frozen oz lb. Package, Ground Beef Substitute, 16 Ounce', 9.29, 4.6)]
[(1, 1, 1, 'enthuz', 'June 9, 2021', 'Toxic chemical plant', 3.0), (2, 2, 1, 'marc archambeau', 'June 11, 2021', "This order showed up RANCID! and NO RETURN??????  BS! $70 down the toilet and I can't get a refund for this nasty stuff!  I am so VERY", 29.0), (3, 2, 5, 'Irish Rogue', 'June 5, 2021', 'Great taste better th', 4.0), (4, 2, 5, 'Walter White', 'May 27, 2021', "For recipes like dirty rice, spaghetti, or sloppy Joe's I use Beyond. For actual burgers, I like Impossible better. This makes a better crumble. But it's so gross to cook. It goes from brown to blood red. All that heme protein is kind of creepy. But the taste is nice. The texture is OK. But really take my advice and cook this IN stuff as opposed to using it for b", 71.0), (5, 2,

In [43]:
def drop_all_tables():
    conn = sqlite3.connect("reviewdb.db")
    c = conn.cursor()
    c.execute("""DROP TABLE IF EXISTS brand_table""")
    c.execute("""DROP TABLE IF EXISTS product_table""")
    c.execute("""DROP TABLE IF EXISTS review_table""")


    conn.commit()
    conn.close()
drop_all_tables()

In [ ]:
def write_to_db_old(data):
    conn = sqlite3.connect("reviewdb.db")
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS scraped_reviews ('brand_name', 'product_name', 'asin', 'price', 'overall_rating', 'reviewer_username', 'review_date', 'review_rating', 'review_text')""")
    
    for entry in data:
        c.execute("INSERT INTO scraped_reviews ('brand_name', 'product_name', 'asin', 'price', 'overall_rating', 'reviewer_username', 'review_date', 'review_rating', 'review_text') VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                  (entry[0], entry[1], entry[2], entry[3], entry[4], entry[5], entry[6], entry[7], entry[8]))
        
    conn.commit()
    conn.close()